<a href="https://colab.research.google.com/github/hassanykb/Urban-form-Accra/blob/main/Urban_forms_accra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install matplotlib_scalebar

In [5]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
from sklearn.ensemble import RandomForestClassifier
import shap
import contextily as cx
from matplotlib_scalebar.scalebar import ScaleBar

In [6]:
# Load data
df = gpd.read_file('/content/drive/MyDrive/Urban energy analysis/UF_reduced.gpkg')
form_data = df.copy()
form_data.fillna(0, inplace=True)
form_data.head()

,id,Num_Build,Accessibil,Intersecti,Build_heig,B_volume,Pop_dens,geometry
0,4748.0,2.0,0.0,18.0,2.9,1296.0,0.0000,"MULTIPOLYGON (((775654.885 630960.811, 775690...."
1,4748.0,2.0,0.0,18.0,2.5,1296.0,0.0000,"MULTIPOLYGON (((775654.885 630960.811, 775690...."
2,4748.0,12.0,0.0,18.0,2.5,2795.0,8.7322,"MULTIPOLYGON (((775654.885 630960.811, 775690...."
3,4748.0,12.0,0.0,18.0,2.0,2795.0,8.7322,"MULTIPOLYGON (((775654.885 630960.811, 775690...."
4,4748.0,2.0,0.0,18.0,2.9,930.0,0.0000,"MULTIPOLYGON (((775654.885 630960.811, 775690...."


In [7]:
# Standardize the data
scaler = StandardScaler()
form_data[['Num_Build_T', 'Accessibil_T', 'Intersecti_T', 'Build_heig_T', 'B_volume_T', 'Pop_dens_T']] = scaler.fit_transform(
    form_data[['Num_Build', 'Accessibil', 'Intersecti', 'Build_heig', 'B_volume', 'Pop_dens']]
)

In [8]:
# Remove outliers using Z-score
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

In [9]:
# Columns to cluster
cluster_columns = ['Num_Build_T', 'Accessibil_T', 'Intersecti_T', 'Build_heig_T', 'B_volume_T', 'Pop_dens_T']

# Remove outliers
form_data_clean = remove_outliers(form_data, cluster_columns)
print(f"Original rows: {len(form_data)}, Rows after outlier removal: {len(form_data_clean)}")

Original rows: 2233307, Rows after outlier removal: 1537918


In [10]:
# K-means clustering
kmeans = KMeans(n_clusters=4, n_init=10, random_state=42)
kmeans.fit(form_data_clean[cluster_columns])

KMeans(n_clusters=4, n_init=10, random_state=42)

In [11]:
# Assign and sort cluster labels
centroids = kmeans.cluster_centers_
sorted_indices = np.argsort(centroids[:, 5])  # Sort by Pop_dens_T
label_mapping = {old: new for new, old in enumerate(sorted_indices)}
form_data_clean['kmeans_4'] = kmeans.labels_.copy()
form_data_clean['kmeans_4'] = form_data_clean['kmeans_4'].map(label_mapping)

In [ ]:
# --- SHAP Integration ---
# Train a Random Forest classifier to predict cluster labels
X = form_data_clean[cluster_columns]  # Features
y = form_data_clean['kmeans_4']       # Cluster labels as pseudo-targets
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

In [ ]:
# Use SHAP to explain the Random Forest predictions
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X)

In [ ]:
# Summary plot: Feature importance across all clusters
plt.figure()
shap.summary_plot(shap_values, X, plot_type="bar", class_names=['UF1', 'UF2', 'UF3', 'UF4'])
plt.title("SHAP Feature Importance for Cluster Assignment")
plt.show()

In [ ]:
# Detailed summary plot: Feature impact direction and magnitude
plt.figure()
shap.summary_plot(shap_values, X, feature_names=cluster_columns, class_names=['UF1', 'UF2', 'UF3', 'UF4'])
plt.title("SHAP Summary Plot for Cluster Assignment")
plt.show()

In [ ]:
# Dependence plot for a specific feature (e.g., Pop_dens_T)
plt.figure()
shap.dependence_plot(5, shap_values[0], X, feature_names=cluster_columns)  # 5 = Pop_dens_T index
plt.title("SHAP Dependence Plot for Pop_dens_T (UF1)")
plt.show()

In [ ]:
# --- Plotting Clusters (unchanged from before) ---
custom_colors = ['#E6F0FA', '#A3CFFA', '#4A90E2', '#1C3F7C']
custom_cmap = ListedColormap(custom_colors)

cluster_map = form_data_clean.plot(
    column='kmeans_4',
    legend=True,
    figsize=(20, 10),
    categorical=True,
    cmap=custom_cmap,
    legend_kwds={
        'loc': 'lower right',
        'markerscale': 1.29,
        'title_fontsize': 'medium',
        'fontsize': 'small',
        'labels': ['UF1', 'UF2', 'UF3', 'UF4']
    }
)
cx.add_basemap(cluster_map, crs=form_data_clean.crs, alpha=0.45)
scalebar = ScaleBar(dx=1, units="m", location="lower left")
cluster_map.add_artist(scalebar)

def add_north_arrow(ax, scale=0.5, xlim_pos=0.9025, ylim_pos=0.765, color='#000', text_scaler=2, text_yT=-1.25):
    ax.text(xlim_pos, ylim_pos, 'N', transform=ax.transAxes, fontsize=12 * scale, color=color, weight='bold')

add_north_arrow(cluster_map, scale=.5, xlim_pos=.9025, ylim_pos=.765, color='#000', text_scaler=2, text_yT=-1.25)

plt.show()